In [2]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os

# 设置数据路径
load_path = "saved_scans/"  # 修改为你的实际路径
files = sorted([f for f in os.listdir(load_path) if f.endswith(".npz")])

# 设置 colormap
colormap = plt.get_cmap('jet')

# 创建坐标轴
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)

# 创建极坐标圆
def create_polar_rings(radii, z_height=0.0, resolution=100):
    rings = []
    for r in radii:
        theta = np.linspace(0, 2 * np.pi, resolution)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = np.full_like(x, z_height)
        pts = np.stack([x, y, z], axis=1)
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(pts)
        lines = [[i, (i+1)%resolution] for i in range(resolution)]
        line_set.lines = o3d.utility.Vector2iVector(lines)
        colors = [[0.5, 0.5, 0.5] for _ in lines]
        line_set.colors = o3d.utility.Vector3dVector(colors)
        rings.append(line_set)
    return rings

polar_rings = create_polar_rings([1.0, 2.0, 3.0])  # 半径可自调


for file in files:
    # 读取数据
    data = np.load(os.path.join(load_path, file))
    xyz = data['xyz']
    intensity = data['intensity']
    
    # 归一化 intensity
    intensity_norm = (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity) + 1e-6)
    colors = colormap(intensity_norm)[:, :3]  # 去掉 alpha 通道

    # 创建 Open3D 点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    

    # 显示点云
    print(f"Showing: {file}")
    o3d.visualization.draw_geometries([pcd, axis] + polar_rings, window_name=file)

    # 可选：按键跳过或结束
    user_input = input("Press Enter to see next, or type 'q' to quit: ")
    if user_input.lower() == 'q':
        break


Showing: 0_points_raw_1749546707.npz


In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import os

# 设置数据路径
load_path = "saved_scans/"
files = sorted([f for f in os.listdir(load_path) if f.endswith(".npz")])

# 设置 colormap
colormap = plt.get_cmap('jet')

# 创建坐标轴
axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.01)

# 创建极坐标圆
def create_polar_rings(radii, z_height=0.0, resolution=100):
    rings = []
    for r in radii:
        theta = np.linspace(0, 2 * np.pi, resolution)
        x = r * np.cos(theta)
        y = r * np.sin(theta)
        z = np.full_like(x, z_height)
        pts = np.stack([x, y, z], axis=1)
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(pts)
        lines = [[i, (i+1)%resolution] for i in range(resolution)]
        line_set.lines = o3d.utility.Vector2iVector(lines)
        colors = [[0.5, 0.5, 0.5] for _ in lines]
        line_set.colors = o3d.utility.Vector3dVector(colors)
        rings.append(line_set)
    return rings

polar_rings = create_polar_rings([0.5, 1.0, 1.5, 2.0, 3.0])  # 半径列表

# 遍历所有文件
for file in files:
    print(f"Loading: {file}")
    data = np.load(os.path.join(load_path, file))
    
    # 加载字段
    xyz = data['xyz']
    intensity = data['intensity']
    r = data['r']  # 必须含有 r 字段

    # 筛选极径在 0.5 ~ 1.5 米之间的点
    mask_r = (r >= 0.4) & (r <= 1.3)
    xyz = xyz[mask_r]
    intensity = intensity[mask_r]

    # 归一化 intensity
    if len(intensity) == 0:
        print("No points in valid range, skipping...")
        continue
    intensity_norm = (intensity - np.min(intensity)) / (np.max(intensity) - np.min(intensity) + 1e-6)
    colors = colormap(intensity_norm)[:, :3]

    # 构造点云对象
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # 显示点云、坐标轴和极坐标环
    print(f"Showing: {file}, Points: {len(xyz)}")
    o3d.visualization.draw_geometries([pcd, axis] + polar_rings, window_name=file)

    # 用户操作
    user_input = input("Press Enter to see next, or type 'q' to quit: ")
    if user_input.lower() == 'q':
        break


Loading: pm_points_raw_1749545740.npz


KeyError: 'xyz is not a file in the archive'

In [25]:
import numpy as np
import open3d as o3d
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import os, gc

# 设置路径
load_path = "saved_scans/"
filename = "45_points_raw_1749547297.npz"
filepath = os.path.join(load_path, filename)

# 聚类参数
#DBSCAN_EPS = 0.025         #平面
#DBSCAN_EPS = 0.021         #0度
DBSCAN_EPS = 0.02          #45度

MIN_SAMPLES = 100
Z_SCALE = 0.2

# 极径筛选范围
#R_MIN = 0.4
#R_MAX = 1.4

R_MIN = 0.4
R_MAX = 0.9

# 加载数据
data = np.load(filepath)

# 筛选
mask_r = (data['r'] >= R_MIN) & (data['r'] <= R_MAX)

xyz_full = data['xyz'][mask_r].astype(np.float32)
intensity_full = data['intensity'][mask_r].astype(np.float32)
r = data['r'][mask_r].astype(np.float32)
ts = data['ts'][mask_r]
ch = data['ch'][mask_r]
theta = data['theta'][mask_r]
gc.collect()

# z缩放后的xyz，用于聚类
xyz_scaled = xyz_full.copy()
xyz_scaled[:, 2] *= Z_SCALE

# DBSCAN聚类
db = DBSCAN(eps=DBSCAN_EPS, min_samples=MIN_SAMPLES).fit(xyz_scaled)
labels = db.labels_
unique_labels = set(labels)
print(f"簇数量（不含噪声）: {len(unique_labels) - (1 if -1 in labels else 0)}")
print(f"噪声点数量: {np.sum(labels == -1)}")

# 给每个簇生成颜色，噪声点单独黑色
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
cmap = plt.get_cmap('tab20')  # 20色循环使用
colors = np.zeros((len(xyz_full), 3), dtype=np.float32)

for label in unique_labels:
    if label == -1:
        # 噪声点黑色
        colors[labels == label] = [0, 0, 0]
    else:
        color = cmap(label % 20)[:3]  # 获取RGB，忽略alpha
        colors[labels == label] = color

# 构造点云
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz_full)
pcd.colors = o3d.utility.Vector3dVector(colors)

# 生成坐标轴辅助线（长度1米）
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# 显示
print(f"显示点数：{len(xyz_full)}")
o3d.visualization.draw_geometries([pcd, axes], window_name="DBSCAN聚类结果（不同颜色为不同簇，黑色为噪声）")



簇数量（不含噪声）: 1
噪声点数量: 305
显示点数：4531


In [26]:
# 保存最大两个簇
save_dir = "clusterdata"
os.makedirs(save_dir, exist_ok=True)
save_file = os.path.join(save_dir, filename)

num_clusters_to_save = 1

# 过滤掉噪声点
unique_labels_arr, counts = np.unique(labels[labels != -1], return_counts=True)

if len(unique_labels_arr) < num_clusters_to_save:
    print(f"聚类结果不足 {num_clusters_to_save} 个簇，实际为 {len(unique_labels_arr)} 个。")
else:
    top_indices = np.argsort(counts)[-num_clusters_to_save:][::-1]
    top_labels = unique_labels_arr[top_indices]

    print(f"将保存的 {num_clusters_to_save} 个最大簇标签：{top_labels}")
    print(f"对应点数：{counts[top_indices]}")

    save_dict = {}
    for i, label in enumerate(top_labels, start=1):
        mask = (labels == label)
        prefix = f"cluster{i}"
        save_dict[f"{prefix}_xyz"] = xyz_full[mask]
        save_dict[f"{prefix}_intensity"] = intensity_full[mask]
        save_dict[f"{prefix}_ts"] = ts[mask]
        save_dict[f"{prefix}_ch"] = ch[mask]
        save_dict[f"{prefix}_r"] = r[mask]
        save_dict[f"{prefix}_theta"] = theta[mask]

    np.savez(save_file, **save_dict)
    print(f"已保存前 {num_clusters_to_save} 个最大类的数据到：{save_file}")

将保存的 1 个最大簇标签：[0]
对应点数：[4226]
已保存前 1 个最大类的数据到：clusterdata\45_points_raw_1749547297.npz


In [28]:
import numpy as np
import open3d as o3d
import os

# 读取文件
load_path = "clusterdata/"
filename = "45_points_raw_1749547197.npz"
filepath = os.path.join(load_path, filename)

# 加载数据
data = np.load(filepath)

# 构造 Open3D 点云列表
pcds = []
colors = [[1, 0, 0], [0, 1, 0]]  # 红色和绿色表示两个类

for i in [1, 2]:
    key_xyz = f"cluster{i}_xyz"
    if key_xyz not in data:
        print(f"未找到 {key_xyz}，跳过。")
        continue

    xyz = data[key_xyz]
    color = np.tile(colors[i - 1], (xyz.shape[0], 1))

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(color)
    pcds.append(pcd)

# 添加坐标轴辅助线
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])

# 可视化
print(f"展示两个最大簇，共加载 {len(pcds)} 个点云对象")
o3d.visualization.draw_geometries(pcds + [axes], window_name="两个最大聚类簇（红/绿）")


未找到 cluster2_xyz，跳过。
展示两个最大簇，共加载 1 个点云对象
